In [16]:
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from nltk.corpus import stopwords
import re

In [6]:
with open('/home/mrladidadi/projects/local/DAIC_glossary_generator/big_data/wiki_master_dataframe.pkl', 'rb') as f:
    corpus = pickle.load(f)

In [7]:
count = 0
for i in corpus:
    temp_arr = []
    for j in i['kw']:
        if i['kw'][j] == 'P' or i['kw'][j] == 'K':
            temp_arr.append(j)
    corpus[count]['kw'] = temp_arr
    count += 1

In [8]:
def clean_text(test_str):
    
    if test_str.find('Version history') != -1:
        result = [i for i in range(len(test_str)) if test_str.startswith('Version history', i)] 
        test_str = test_str[:result[-1]]
    elif test_str.find('See also') != -1:
        result = [i for i in range(len(test_str)) if test_str.startswith('See also', i)] 
        test_str = test_str[:result[-1]]
    elif test_str.find('References') != -1:
        result = [i for i in range(len(test_str)) if test_str.startswith('References', i)] 
        test_str = test_str[:result[-1]]
        
        
    clean_text = re.sub("[\[\(].*?[\]\)]", "", test_str).replace('\n','')
    final = ''
    for i in clean_text:
        if i.isalpha() or i.isdigit() or i=='-' or i==' ' or i=='.':
            final += i
            
    
    for i in range(len(final)-2):
        if (final[i].isalpha() and final[i].islower() and final[i]!=' ') and (final[i+1] == '.') and (final[i+2].isalpha() and final[i+2].isupper()):
            final = final[:i+1] + '. ' + final[i+2:]
            
    final = re.sub(' +', ' ', final).replace(' .','.').strip()
    
    return final

In [9]:
def consec_alpha(candidate_string):
    for i in range(0,len(candidate_string)-1):
        if candidate_string[i].isalpha() and candidate_string[i+1].isalpha():
            return True
    return False

In [10]:
def remove_equa(curpage):
    final = ''
    for i in sent_tokenize(curpage):
        if consec_alpha(i)==True and i.find('displaystyle')==-1:
            final += i + ' '
    return final

In [11]:
def custom_clean(curpage, stopword_bool):
    curpage = remove_equa(curpage)
    curpage = clean_text(curpage)
    if stopword_bool == True:
        curpage = stopword_remove(curpage)
    return curpage

In [17]:
count = 0
for i in range(len(corpus)):
    count += 1
    print(str(count)+'/'+str(len(corpus)),end='\r')
    corpus[i]['text'] = custom_clean(corpus[i]['text'], True)
    temp_kw_arr = []
    for j in corpus[i]['kw']:
        if corpus[i]['text'].find(j)!=-1:
            temp_kw_arr.append(j)
    corpus[i]['kw'] = temp_kw_arr

NameError: name 'stopword_remove' is not defined

In [77]:
 with open('/home/mrladidadi/projects/local/DAIC_glossary_generator/big_data/wiki_master_dataframe_cleaned_stopwordrem.pkl', 'wb') as f:
    pickle.dump(corpus, f)